In [1]:
import pandas as pd
import numpy as np

In [2]:
home_credit_default_risk=pd.read_csv('pp_application_train_test_c.csv',sep=';')

In [3]:
home_credit_default_risk.shape

(307507, 797)

### Vérification des valeurs nulls

In [4]:
home_credit_default_risk.isna().sum()

SK_ID_CURR                               0
TARGET                                   0
CODE_GENDER                              0
FLAG_OWN_CAR                             0
FLAG_OWN_REALTY                          0
                                     ...  
CC_NAME_CONTRACT_STATUS_nan_MAX     220602
CC_NAME_CONTRACT_STATUS_nan_MEAN    220602
CC_NAME_CONTRACT_STATUS_nan_SUM     220602
CC_NAME_CONTRACT_STATUS_nan_VAR     221234
CC_COUNT                            220602
Length: 797, dtype: int64

In [5]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [6]:
missing_values = missing_values_table(home_credit_default_risk)
missing_values.head(10)

Your selected dataframe has 797 columns.
There are 616 columns that have missing values.


,Missing Values,% of Total Values
REFUSED_RATE_DOWN_PAYMENT_MEAN,262339,85.3
REFUSED_RATE_DOWN_PAYMENT_MAX,262339,85.3
REFUSED_RATE_DOWN_PAYMENT_MIN,262339,85.3
REFUSED_AMT_DOWN_PAYMENT_MEAN,262339,85.3
REFUSED_AMT_DOWN_PAYMENT_MAX,262339,85.3
REFUSED_AMT_DOWN_PAYMENT_MIN,262339,85.3
REFUSED_APP_CREDIT_PERC_VAR,258350,84.0
CC_AMT_PAYMENT_CURRENT_VAR,246888,80.3
CC_AMT_DRAWINGS_OTHER_CURRENT_VAR,246814,80.3
CC_AMT_DRAWINGS_ATM_CURRENT_VAR,246814,80.3


In [7]:
home_credit_default_risk['REFUSED_RATE_DOWN_PAYMENT_MEAN'].unique()

array([       nan, 0.04287377, 0.        , ..., 0.1089259 , 0.11181011,
       0.08765319])

In [8]:
home_credit_default_risk['REFUSED_RATE_DOWN_PAYMENT_MAX'].unique()

array([       nan, 0.21091948, 0.        , ..., 0.10891886, 0.11181011,
       0.08765319])

In [9]:
home_credit_default_risk['REFUSED_APP_CREDIT_PERC_VAR'].unique()

array([       nan, 0.00522749, 0.        , ..., 0.00411567, 0.01514609,
       0.00704428])

#### Vérification des colonnes avec une seule valeur unique

In [10]:
single_value_columns = [col for col in home_credit_default_risk.columns if home_credit_default_risk[col].nunique() == 1]

print(single_value_columns)
print(len(single_value_columns))

['BURO_CREDIT_ACTIVE_nan_MEAN', 'BURO_CREDIT_CURRENCY_nan_MEAN', 'BURO_CREDIT_TYPE_nan_MEAN', 'BURO_STATUS_nan_MEAN_MEAN', 'PREV_NAME_CONTRACT_TYPE_nan_MEAN', 'PREV_WEEKDAY_APPR_PROCESS_START_nan_MEAN', 'PREV_FLAG_LAST_APPL_PER_CONTRACT_nan_MEAN', 'PREV_NAME_CASH_LOAN_PURPOSE_nan_MEAN', 'PREV_NAME_CONTRACT_STATUS_nan_MEAN', 'PREV_NAME_PAYMENT_TYPE_nan_MEAN', 'PREV_CODE_REJECT_REASON_nan_MEAN', 'PREV_NAME_CLIENT_TYPE_nan_MEAN', 'PREV_NAME_GOODS_CATEGORY_House Construction_MEAN', 'PREV_NAME_GOODS_CATEGORY_nan_MEAN', 'PREV_NAME_PORTFOLIO_nan_MEAN', 'PREV_NAME_PRODUCT_TYPE_nan_MEAN', 'PREV_CHANNEL_TYPE_nan_MEAN', 'PREV_NAME_SELLER_INDUSTRY_nan_MEAN', 'PREV_NAME_YIELD_GROUP_nan_MEAN', 'POS_NAME_CONTRACT_STATUS_nan_MEAN', 'CC_SK_DPD_MIN', 'CC_SK_DPD_DEF_MIN', 'CC_NAME_CONTRACT_STATUS_Approved_MIN', 'CC_NAME_CONTRACT_STATUS_Demand_MIN', 'CC_NAME_CONTRACT_STATUS_Refused_MIN', 'CC_NAME_CONTRACT_STATUS_Sent proposal_MIN', 'CC_NAME_CONTRACT_STATUS_nan_MIN', 'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_

In [11]:
home_credit_default_risk.drop(columns=single_value_columns, inplace=True)

In [12]:
home_credit_default_risk.shape

(307507, 766)

#### Vérification des valeurs infinies

In [13]:
print((home_credit_default_risk == np.inf).sum().sum()) 

20


In [14]:
print((home_credit_default_risk == -np.inf).sum().sum())

0


In [15]:
# Remplacer les valeurs infinies par NaN
home_credit_default_risk.replace([np.inf, -np.inf], np.nan, inplace=True)

# Pour vérifier que les valeurs infinies ont bien été remplacées
print((home_credit_default_risk == np.inf).sum().sum()) 

0


In [16]:
#Imputation pour les colonnes numériques 
home_credit_default_risk_new = home_credit_default_risk.fillna(home_credit_default_risk.mean())
# Imputation pour les colonnes catégoriques 
home_credit_default_risk_new = home_credit_default_risk.fillna(home_credit_default_risk.mode().iloc[0])

In [17]:
missing_values = missing_values_table(home_credit_default_risk_new)
missing_values.head(40)

Your selected dataframe has 766 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [27]:
from scipy.stats import chi2_contingency
y = home_credit_default_risk['TARGET']

chi_ls = []

for feature in home_credit_default_risk.select_dtypes(include='object'):
    
    # create contingency table
    c = pd.crosstab(y, home_credit_default_risk[feature])
    
    # chi_test
    p_value = chi2_contingency(c)[1]
    chi_ls.append(p_value)

In [28]:
sns.set(style="ticks", rc={'figure.figsize': (10, 10)}, palette="BrBG")

df_pvalue = pd.DataFrame(chi_ls, index=home_credit_default_risk.select_dtypes(include='object').columns, columns=['p_value']).sort_values(by='p_value', ascending=True)

pd.set_option('display.float_format', '{:,.3f}'.format)
print(df_pvalue)

df_pvalue.plot.barh()
plt.xlabel('p value')
plt.title('Feature importance based on chi-square test')
plt.show()

Empty DataFrame
Columns: [p_value]
Index: []


TypeError: no numeric data to plot

In [26]:
 home_credit_default_risk_new.to_csv('pp_application_train_test_n.csv', sep=';',index=False, header=True)